In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Importing the Dataset

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 91% 74.0M/80.9M [00:00<00:00, 164MB/s]
100% 80.9M/80.9M [00:00<00:00, 131MB/s]


In [ ]:
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"

with ZipFile(dataset, 'r') as zip:
  zip.extractall()

# Importing Dependencies

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Processing

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')
twitter_data.shape

(1599999, 6)

In [ ]:
cols = ['target', 'id', 'date', 'flat', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = cols, encoding = 'ISO-8859-1')
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flat,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().any().any()

False

In [ ]:
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace = True)

In [ ]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

# Stemming

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flat,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


# Splitting Train and Test Data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 42)

In [ ]:
print("Shape of X: ", X.shape)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

print("Shape of Y: ", Y.shape)
print("Shape of Y_train: ", Y_train.shape)
print("Shape of Y_test: ", Y_test.shape)

Shape of X:  (1600000,)
Shape of X_train:  (1280000,)
Shape of X_test:  (320000,)
Shape of Y:  (1600000,)
Shape of Y_train:  (1280000,)
Shape of Y_test:  (320000,)


In [ ]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 154767)	0.2697660704325824
  (0, 388138)	0.20555120011808467
  (0, 220296)	0.4301567790762487
  (0, 406297)	0.2978221095272138
  (0, 286478)	0.16123218610004275
  (0, 454381)	0.20169626473577718
  (0, 205794)	0.2414022906380175
  (0, 124524)	0.18318401951949756
  (0, 4832)	0.31707426786115905
  (0, 128605)	0.22108856600702775
  (0, 175252)	0.22407080547034602
  (0, 146067)	0.12929728405657018
  (0, 239679)	0.15130037108228483
  (0, 307108)	0.4620604881532448
  (1, 124611)	0.5113765148324884
  (1, 161801)	0.5778049407933611
  (1, 445870)	0.6361096685891185
  (2, 400192)	0.2722541116865256
  (2, 31063)	0.1936303169258752
  (2, 78861)	0.21039643874061958
  (2, 312657)	0.3154702974657607
  (2, 453420)	0.2347069337186747
  (2, 12436)	0.2529872032123258
  (2, 267649)	0.19309660201644555
  (2, 358186)	0.19837942712286838
  :	:
  (1279997, 120136)	0.5183864288132355
  (1279997, 80839)	0.4085075096529922
  (1279997, 262076)	0.3257051331036561
  (1279997, 372797)	0.3021907911926725
  (1279

In [ ]:
print(X_test)

  (0, 453357)	0.1781708363247895
  (0, 435956)	0.14183025329879742
  (0, 384697)	0.3281164007446601
  (0, 335577)	0.22602158147814247
  (0, 334643)	0.14719329779308424
  (0, 240451)	0.15341308097014625
  (0, 189057)	0.31324918577405797
  (0, 129417)	0.25650960779862714
  (0, 97585)	0.4019235611854435
  (0, 96399)	0.255967788489452
  (0, 78636)	0.5158100011206617
  (0, 43712)	0.23562815302828183
  (0, 28874)	0.1778395103911245
  (1, 438137)	0.16390196841168053
  (1, 428475)	0.2850923429146983
  (1, 409742)	0.19539196331452124
  (1, 328541)	0.30370566878650734
  (1, 325414)	0.2832875855026037
  (1, 318909)	0.20246938925608493
  (1, 303003)	0.19949441578138133
  (1, 301683)	0.13212235134015302
  (1, 240223)	0.1674195650536303
  (1, 233854)	0.3852709938491561
  (1, 183279)	0.24586158827112847
  (1, 171245)	0.12468774856570086
  :	:
  (319997, 225096)	0.2933179603743362
  (319997, 169469)	0.19450444826456267
  (319997, 146067)	0.3148589224636577
  (319997, 135536)	0.2180996497753238
  (3199

# Training the ML Model

In [ ]:
model = LogisticRegression(max_iter = 1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

# Model Evaluation

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print("Accuracy Score for training = ", training_data_accuracy)

Accuracy Score for training =  0.81021171875


In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print("Accuracy Score for testing = ", test_data_accuracy)

Accuracy Score for testing =  0.778690625


Test accuracy = 77.86%

# Saving the Model

In [ ]:
import pickle

In [ ]:
filename = 'TSA_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open('/content/TSA_model.sav', 'rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

new_prediction = loaded_model.predict(X_new)
print(new_prediction)

if (new_prediction[0] == 0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[0]
Negative Tweet


In [ ]:
## hyperparameter tuning
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# parameters = {
#     'C': [0.01, 0.1, 1, 10],  # Regularization strength
#     'solver': ['liblinear', 'lbfgs', 'sag', 'saga']  # Optimization algorithm
# }

# Support Vector Machine

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 42)
classifier.fit(X_train, Y_train)

In [ ]:
Y_cpred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_cpred)
print(cm)
accuracy_score(Y_test,Y_cpred)

In [ ]:
filename = 'TSA_classifier.sav'
pickle.dump(model, open(filename, 'wb'))

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Build a Gaussian Classifier
NBmodel = GaussianNB()

# Model training
NBmodel.fit(X_train, Y_train)

# Predict Output
predicted = NBmodel.predict([X_test[6]])

print("Actual Value:", Y_test[6])
print("Predicted Value:", predicted[0])

In [ ]:
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    f1_score,
)

NBY_pred = NBmodel.predict(X_test)
NBaccuracy = accuracy_score(NBY_pred, Y_test)
f1 = f1_score(Y_pred, Y_test, average="weighted")

print("Accuracy:", NBaccuracy)
print("F1 Score:", f1)

In [ ]:
NBcm = confusion_matrix(Y_test, NBY_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=NBcm)
disp.plot();

In [ ]:
filename = 'TSA_NBmodel.sav'
pickle.dump(model, open(filename, 'wb'))